# 22 - Mirrors and Modules


Doing an example tutorial for example brought up in Issue #372

![Mirror and Module Combo](../images_wiki/AdvancedJournals/22_mirror_moduleCombo.PNG)



In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_22')
if not os.path.exists(testfolder):
    os.makedirs(testfolder)
        
print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_22


In [2]:
import bifacial_radiance
import numpy as np
import pprint   
import pandas as pd

<a id='step2'></a>

In [3]:
demo = bifacial_radiance.RadianceObj('tutorial_22', path=testfolder)  # Adding a simulation name. This is optional.
demo.setGround(0.2) 
epwfile = demo.getEPW(lat=37.5, lon=-77.6) 
metdata = demo.readWeatherFile(weatherFile=epwfile, coerce_year=2021) 
timeindex = metdata.datetime.index(pd.to_datetime('2021-01-01 12:0:0 -5'))
demo.gendaylit(timeindex) # Choosing a december time when the sun is lower in the horizon


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_22
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


'skies\\sky2_37.5_-77.33_2021-01-01_1200.rad'

## 1. Create your module and evaluate irradiance without the mirror element

In [4]:
tilt = 75
sceneDict1 = {'tilt':tilt,'pitch':5,'clearance_height':0.05,'azimuth':180, 
              'nMods': 1, 'nRows': 1, 'originx': 0, 'originy': 0, 'appendRadfile':True} 
mymodule1 = demo.makeModule(name='test-module',x=2,y=1, numpanels=1)
sceneObj1 = demo.makeScene(mymodule1, sceneDict1)  


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



In [5]:
octfile = demo.makeOct(demo.getfilelist())  
analysis = bifacial_radiance.AnalysisObj(octfile, demo.basename)
frontscan, backscan = analysis.moduleAnalysis(sceneObj1, sensorsy=1)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  

Created tutorial_22.oct
Linescan in process: tutorial_22_Front
Linescan in process: tutorial_22_Back
Saved: results\irr_tutorial_22.csv


In [6]:
withoutMirror = bifacial_radiance.load.read1Result('results\irr_tutorial_22.csv')
withoutMirror

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,2.484128e-18,-0.020284,0.538398,0.532704,a0.0.a0.test-module.6457,a0.0.a0.test-module.2310,140.0624,65.60773,0.468415


## 2. Add Mirror

### Approach 1: Pretend the mirror is another module.

We start by creating the mirror material in our ground.rad file, in case it is not there. For mirror or glass primitives (material classes), pecularity and roughness are not needed. 

You could alternatively do a plastic material, and increase the specularity and lower the roughness to get a very reflective surface.

In [7]:
demo.addMaterial(material='testmirror', Rrefl=0.94, Grefl=0.96, Brefl=0.96, 
         materialtype = 'mirror') # specularity and roughness not needed for mirrors or glass. 

Material exists, overwriting...

Added material testmirror to file ground.rad


In [8]:
mymodule2 = demo.makeModule(name='test-mirror',x=2,y=1, numpanels=1, modulematerial='testmirror')


Module Name: test-mirror
Module test-mirror updated in module.json
Pre-existing .rad file objects\test-mirror.rad will be overwritten



We calculate the displacement of the morrir as per the equations show in the image at the beginning of the tutorial 

In [9]:
originy = -(0.5*mymodule2.sceney + 0.5*mymodule1.sceney*np.cos(np.radians(tilt)))

In [10]:
sceneDict2 = {'tilt':0,'pitch':0.00001,'clearance_height':0.05,'azimuth':180, 
              'nMods': 1, 'nRows': 1, 'originx': 0, 'originy': originy, 'appendRadfile':True} 
sceneObj2 = demo.makeScene(mymodule2, sceneDict2)  

In [11]:
octfile = demo.makeOct(demo.getfilelist())  

Created tutorial_22.oct


Use rvu in the terminal or by commenting out the cell below to view the generated geometry, it should look like this:
    
![Mirror and Module Combo](../images_wiki/AdvancedJournals/22_mirror_moduleCombo_rvu.PNG)



In [12]:
## Comment the line below to run rvu from the Jupyter notebook instead of your terminal.
## Simulation will stop until you close the rvu window

# !rvu -vf views\front.vp -e .01 -vp 4 -0.6 1 -vd -0.9939 0.1104 0.0 tutorial_22.oct

In [13]:
analysis = bifacial_radiance.AnalysisObj(octfile, demo.basename)
frontscan, backscan = analysis.moduleAnalysis(sceneObj1, sensorsy=1)
results = analysis.analysis(octfile, name=demo.basename+'_withMirror', frontscan=frontscan, backscan=backscan)  
withMirror = bifacial_radiance.load.read1Result('results\irr_tutorial_22_withMirror.csv')
withMirror

Linescan in process: tutorial_22_withMirror_Front
Linescan in process: tutorial_22_withMirror_Back
Saved: results\irr_tutorial_22_withMirror.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,2.484128e-18,-0.020284,0.538398,0.532704,a0.0.a0.test-module.6457,a0.0.a0.test-module.2310,171.875133,64.659397,0.376198


Just as a sanity check, we could sample the mirror...

In [14]:
frontscan, backscan = analysis.moduleAnalysis(sceneObj2, sensorsy=1)
results = analysis.analysis(octfile, name=demo.basename+'_Mirroritself', frontscan=frontscan, backscan=backscan)  
bifacial_radiance.load.read1Result('results\irr_tutorial_22_Mirroritself.csv')

Linescan in process: tutorial_22_Mirroritself_Front
Linescan in process: tutorial_22_Mirroritself_Back
Saved: results\irr_tutorial_22_Mirroritself.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.0,-0.629409,0.071,0.049,a0.0.a0.test-mirror.6457,a0.0.a0.test-mirror.2310,160.3301,0.375643,0.002343


And we can calculate the increase in front irradiance from the mirror:

In [15]:
print("Gain from mirror:", round((withMirror.Wm2Front[0] - withoutMirror.Wm2Front[0] )*100/withoutMirror.Wm2Front[0],1 ), "%" )

Gain from mirror: 22.7 %


### Approach 2: 

Create mirrors as their own objects and Append to Scene, like on tutorial 5. Sample code below:

In [16]:
# name='Mirror1'
# text='! genbox black cuteMirror 2 1 0.02 | xform -t -1 -0.5 0 -t 0 {} 0'.format(originy)
# customObject = demo.makeCustomObject(name,text)
# demo.appendtoScene(radfile=scene.radfiles, customObject=customObject, text="!xform -rz 0")
# sceneObj2 = demo.makeScene(mymodule2, sceneDict2)  

and then you do your Scene, Oct, and Analysis as usual.